#  Task: Extract Handwritten data from a Bank form

We have been given a 2 page long form that a user has to fill, we need to extract this data and convert this into text using
HTR tehcniques. The following notebook uses a series of steps that have been undertaken for the same.

1 Convert 2 page pdf to images.

2 Warp each image to bring the input images to standard format.

3 Make a mask that contains all the ROI, Build a script to get the coordinates.
4 Place the mask over the image and extract ROIs (Done)

5 For each ROI split the image into constituent characters. 

6 Build a Text Classifier using a dataset

7 Preprocess and Train the CNN.

8 Build a function that takes input and returns the text data

(Note: for now we assume the image is scanned and hence warping can be skipped, shall be taken up later
PDF to Image library is complex to load on windows machine, hence will be taken up in google colab later onn.)

In [1]:
# Importing the basic libraries
import cv2
import numpy as np
from keras.models import load_model

In [2]:
imgInput = cv2.imread('form-2.jpg')
imgshow = imgInput.copy()
img = cv2.imread('page1.jpeg',0)
query = cv2.imread('form-1.jpg',0)
mdl = load_model('htr.h5')

In [3]:
h, w, _ = imgInput.shape

In [6]:
# Declaring variables to be used
cnt , p1 , p2 , checkroi , num , textroi , dateroi  = 0 , [] , [] , [] , 0 , [] , []

# Get ROI regions from image

In [7]:
# Script to get ROIs
# Here we have divided each image into 2 parts and taking manual roi one after the other
# Long Long Task
def click_event(event, x, y, flags, params): 
        global cnt,p1,p2,textroi,num,checkroi,dateroi
        if event == cv2.EVENT_LBUTTONDOWN: 
            if cnt ==0:
                p1 = [x*2,(y)*2]
                cnt += 1
                cv2.imshow('image', img[:600,:]) 
            else:
                p2 = [x*2,(y)*2]
                cnt -= 1
                name = input("Enter Name: ")
                dtp = input("Enter Type: ")
                if dtp=='check':
                    checkroi.append([p1,p2,name])
                elif dtp== 'text':
                    num =int(input('No. of Boxes: '))
                    textroi.append([p1,p2,name,num])
                elif dtp == 'date':
                    dateroi.append([p1,p2,name])

In [8]:
img = cv2.imread('form-2.jpg', 1) 
img = cv2.resize(img,(w//2,h//2))
    
while(1):
    # displaying the image 
    cv2.imshow('image', img[:600,:]) 
    # and calling the click_event() function 
    cv2.setMouseCallback('image', click_event) 
    
    # wait for a key to be pressed to exit 
    if cv2.waitKey(1) & 0xFF == ord('z') :
        break

Enter Name: Bangladesh
Enter Type: check
Enter Name: Bhutan
Enter Type: check
Enter Name: Maldives
Enter Type: check
Enter Name: Nepal
Enter Type: check
Enter Name: PK
Enter Type: check
Enter Name: Sri Lanka
Enter Type: check
Enter Name: AAIM
Enter Type: check
Enter Name: Anti Theft
Enter Type: check
Enter Name: Limit TTP
Enter Type: check
Enter Name: Diff Abled Design
Enter Type: check
Enter Name: Usage Restric
Enter Type: check
Enter Name: AAIM No.
Enter Type: text
No. of Boxes: 17
Enter Name: Expiry Dt
Enter Type: date
Enter Name: F. Embassy
Enter Type: check
Enter Name: Racing
Enter Type: check
Enter Name: Driving Tution
Enter Type: check
Enter Name: Vintage Car
Enter Type: check
Enter Name: Fibre Glass Tank
Enter Type: check
Enter Name: Loss Cover
Enter Type: check
Enter Name: Imp w/o Custom duty
Enter Type: check
Enter Name: Payment Adv
Enter Type: check
Enter Name: Vehicle Insp
Enter Type: check
Enter Name: Ren Notice
Enter Type: check
Enter Name: Sale Deed
Enter Type: check
Ent

 For bounding boxes there are 2 categories, the one with bounding boxes and the other with blanks
 For the ones with bounding boexs we need to resize the image to 28x28*n where n is the number of boxes
 For the ones with fields we will have to employ histogram splitting method which will be done in the next stage.

# Image Warping

In [3]:
img = cv2.imread('page1.jpeg',0)    # Page1 of form, change page for next page
query = cv2.imread('form-1.jpg',0)   # Scanned page1

In [4]:
# Image Warping
orb = cv2.ORB_create(10000)
kp1, des1 = orb.detectAndCompute(query,None)
#imkp = cv2.drawKeypoints(query,kp1,None) # if one wants to see the points detected
#cv2.imshow('',imgmt)
#cv2.waitKey(0)
kp2, des2 = orb.detectAndCompute(img,None)
bf = cv2.BFMatcher(cv2.NORM_HAMMING)
matches = bf.match(des2,des1)
matches.sort(key=lambda x:x.distance)
good = matches[:1000]
imgmt = cv2.drawMatches(img,kp2,query,kp1,good,None,flags=2)
srcpt = np.float32([kp2[m.queryIdx].pt for m in good]).reshape(-1,1,2)
dstpt = np.float32([kp1[m.trainIdx].pt for m in good]).reshape(-1,1,2)
M , _ = cv2.findHomography(srcpt,dstpt,cv2.RANSAC,5.0)
imgsc = cv2.warpPerspective(img,M,(1654,2339))

# Processing Functions

In [ ]:
# Cell that takes in text roi and outputs the OCRed data
def text_data(listed):
    dct = {} # Dictionary to contain all fields
    for row in listed:
        # Slicing
        roi = imgsc[row[0][1]:row[1][1],row[0][0]:row[1][0]]
        n = row[3]  # to get data from dates, skip these lines and replace n with 7 
        if(n==0):
            continue
        roi = cv2.resize(roi,(28*n,28)) 
        i = 0
        field = []
        while(i<28*8):
            sliced = roi[:,i:i+28] # splitting into each box
            _, sliced  = cv2.threshold(sliced, 80, 255, cv2.THRESH_BINARY) 
            if(np.sum(sliced)<10000):    # To introduce space if there is no character
                field.append(' ')
            else:
                field.append(get_char(sliced)) # get_char will be the function which will fetch the 
            # predicted value from the CNN
            i += 28   
        cv2.waitKey(0)
        dct[row[2]] = ''.join(field)
    return dct

In [ ]:
# For text data that does not have boxes
def nobox_data(listed):
    for row in listed:
        if row[3]!=0:
            continue
        dct = {}
        img = imgsc[row[0][1]:row[1][1],row[0][0]:row[1][0]]
        img = np.asarray(cv2.bitwise_not(img))
        img = img//255
        arr = [0 for i in range(img.shape[1])]
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                arr[j] += img[i][j]
        count = 0
        for i in range(0,len(arr)):
            if(arr[i]==0):
                count += 1
            if(arr[i]!=0 and count>=5):
                print(i-count,' - ',i)
                count = 0
        # Start of split
        for idx,el in enumerate(arr):
            if el >= 5:
                print(idx)
                break
        idx -= 10
        split = []
        count = 0
        # Find split of 5 pixels or more
        for i in range(idx,len(arr)):
            if(arr[i]<=2):
                count += 1
            elif(arr[i]>0 and count>=1):
                split.append((idx,i-(count//2)))
                count = 0
                idx = i
        l = []
        for idx,el in enumerate(split[1:]):
            img = verticleadjust(er[:,el[0]:el[1]])
            l.append(predict(verticleadjust(er[:,el[1]:])))
        dct[row[2]] = ''.join(l)
    return dct

In [ ]:
# Vertically trimming the image
def verticleadjust(im):
    # Start of split
    img = np.asarray(cv2.bitwise_not(im))
    img = img//255
    ar2 = [0 for i in range(img.shape[0])]
    for j in range(img.shape[1]):
        for i in range(img.shape[0]):
            ar2[i] += img[i][j]
    
    for idx,el in enumerate(ar2):
        if el >= 5:
            break
    idx -= 10
    end = len(ar2)-1
    for i,el in enumerate(ar2):
        if el >= 5:
            end = i
    end += 10
    return cv2.resize(im[idx:end,:],(28,28))

In [ ]:
# Function that takes in date roi and gives output 
# Cell that takes in text roi and outputs the OCRed data
def date_data(listed):
    dct = {} # Dictionary to contain all fields
    for row in listed:
        # Slicing
        roi = imgsc[row[0][1]:row[1][1],row[0][0]:row[1][0]]
        roi = cv2.resize(roi,(28*8,28)) 
        i = 0
        field = []
        while(i<28*8):
            sliced = roi[:,i:i+28] # splitting into each box
            _, sliced  = cv2.threshold(sliced, 80, 255, cv2.THRESH_BINARY) 
            field.append(get_num(sliced)) # get_char will be the function which will fetch the 
            # predicted value from the CNN
            i += 28   
        cv2.waitKey(0)
        dct[row[2]] = str(field[0:2])+'/'+str(field[2:4])+'/'+str(field[4:])
    return dct

In [ ]:
# Function that takes in checkbox data and returns list of checkboxes that were ticked
def checkdata(lst):
    ticked = []
    for d in lst:
        img = imgsc[d[0][1]:d[1][1],d[0][0]:d[1][0]]
        _, th = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY) 
        inv = cv2.bitwise_not(th)
        s = np.sum(inv)
        if(s>25000):
            ticked.append(d[2])
    return ticked

In [ ]:
# Get char function
# Takes in the slice, gets prediction from result and matches it with column correctly
labels = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'J', 11: 'K', 
          12: 'l', 13: 'S', 14: 'T', 15: 'U', 16: 'V', 17: 'W', 18: 'X', 19: 'Y', 20: 'a', 21: 'A', 22: 'b',
          23: 'B', 24: 'c', 25: 'd', 26: 'D',  27: 'e', 28: 'E', 29: 'f', 30: 'g', 31: 'G', 32: 'h',
          33: 'H',  34: 'i', 35: 'L', 36: 'm', 37: 'n', 38: 'N', 39: 'o', 40: 'p', 41: 'q', 42: 'Q', 
          43: 'r', 44: 'R', 45: 'T', 46: 'z'}

def get_char(img):
    img = img/255.0
    img = img[2:-2,2:-2]
    img = img.resize(img,(28,28))
    img = np.reshape(img,(-1,28,28,1))
    pred = mdl.predict(img)
    idx = pred.argmax(axis=-1)
    return labels[idx[0]]

In [ ]:
# Function that takes in date data and predicts the date
def get_num(img):
    img = img/255.0
    img = img[2:-2,2:-2]
    img = img.resize(img,(28,28))
    img = np.reshape(img,(-1,28,28,1))
    pred = mnist.predict(img)
    idx = pred.argmax(axis=-1)
    return idx[0]

The roi data has been stored in the text file, it contains
date1,date2,check1,check2,text1,text2
where date, check and text correspond to the type and 1 and 2 are the page numbers
the lists are to be copied according to need and send to the functions as mentioned in the name.
This has been done to avoid confusion and redundant cells in the notebook.

# Output

In [ ]:
text = text_data(text1) # or text2
text_wo_box = nobox_data(text1) # or text2
dates = date_data(date1) # or date2
checkboxes = check_data(check1) # or check2

Pls. Note- OCR for test without data is being perfected, and due to scanner issues photo was clicked from mobile with introduced substantial noise which caused accuracy to dip, for scanned images the code shall work well. 
Due to upcoming end sem exams, I could not give more time, apologies for that.
Thankyou